In [55]:
import yfinance as yf
import pandas as pd
from pathlib import Path
import math

In [56]:
result_folder = Path.cwd().parent / 'Results'

symbols = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker'].tolist()
symbols = pd.Series(symbols).unique()
backup = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker']
if len(backup) > 0:
    backup.to_csv(result_folder / 'known_tickers.csv', index=False)
owned_tickers = pd.read_csv(result_folder / 'owned_tickers.csv')['Ticker'].tolist()

In [57]:
df = pd.DataFrame(columns=[
    'Ticker', 
    'short_name', 
    'in_portfolio',
    'value_score',  
    'recommendation_score',
    'analyst_mean%', 
    'insider_buy%', 
    'sector', 
    'industry', 
    'P/E (TTM)', 
    'ROA%',
])

for symbol in symbols: 
    ticker = yf.Ticker(symbol)
    try:
        in_portfolio = 'No'
        if symbol in owned_tickers:
            in_portfolio = 'Yes'

        analyst_mean = 0
        if 'mean' in ticker.analyst_price_targets and ticker.info['currentPrice']:
            analyst_mean = ticker.analyst_price_targets['mean'] / ticker.info['currentPrice'] - 1
        
        roa = ticker.info['returnOnAssets']*100
        pe = ticker.info['trailingPE']
        
        value_score = format(2.5 + math.log(roa+5) - math.log(pe+25), '.2f')
        
        insider_buy = 0
        if 'Shares' in ticker.insider_purchases.columns:
            insider_buy = format(ticker.insider_purchases.loc[ticker.insider_purchases.index[4], 'Shares'], '.2f')
            if insider_buy == '<NA>':
                insider_buy = 0

        recommendation_score = format(float(value_score) + analyst_mean * 2.5 + float(insider_buy) * 2.0, '.2f')
        


        df.loc[symbol] = [
            symbol, 
            ticker.info['shortName'], 
            in_portfolio,
            value_score,
            recommendation_score,
            format(analyst_mean * 100, '.2f'), 
            insider_buy,
            ticker.info['sector'], 
            ticker.info['industry'], 
            format(pe, '.2f') if isinstance(pe, (int, float)) else 'N/A', 
            format(roa, '.2f') if isinstance(roa, (int, float)) else 'N/A',
        ]
    except Exception as e:
        print(f"Error with {symbol}: {e}. Removed Ticker from list.")

df.to_csv(result_folder / 'simple_screener_results.csv', index=False)
display(df)

print(f'Got data for {len(df)} stocks')

,Ticker,short_name,in_portfolio,value_score,recommendation_score,analyst_mean%,insider_buy%,sector,industry,P/E (TTM),ROA%
MATAS.CO,MATAS.CO,Matas A/S,No,0.85,1.73,35.14,0,Consumer Cyclical,Specialty Retail,21.11,3.84
TRIFOR.CO,TRIFOR.CO,Trifork Group AG,No,0.75,1.75,40.09,0,Technology,Information Technology Services,12.76,1.59
QQ.L,QQ.L,QINETIQ GROUP PLC ORD 1P,Yes,1.14,1.08,-2.36,0,Industrials,Aerospace & Defense,21.42,6.89
RNMBY,RNMBY,Rheinmetall AG,Yes,0.08,0.08,0.00,0,Industrials,Aerospace & Defense,98.73,5.97
DRS,DRS,"Leonardo DRS, Inc.",No,0.61,1.12,20.20,0.00,Industrials,Aerospace & Defense,39.00,4.64
SAABF,SAABF,Saab AB,Yes,0.39,0.39,0.00,0,Industrials,Aerospace & Defense,46.67,3.70
BCKIY,BCKIY,Babcock International Group Plc,No,1.03,1.03,0.00,0,Industrials,Engineering & Construction,19.04,5.08
BAESY,BAESY,BAE Systems PLC,No,0.83,0.62,-8.35,0,Industrials,Aerospace & Defense,25.05,4.40
IVSO.ST,IVSO.ST,INVISIO AB,No,1.25,0.87,-15.18,0,Technology,Communication Equipment,63.19,20.30
NSKFF,NSKFF,Kongsberg Gruppen ASA,Yes,0.56,0.56,0.00,0,Industrials,Aerospace & Defense,55.98,6.66


Got data for 30 stocks


In [58]:
print(yf.Ticker('RNMBY').info.keys())

dict_keys(['address1', 'city', 'zip', 'country', 'phone', 'fax', 'website', 'industry', 'industryKey', 'industryDisp', 'sector', 'sectorKey', 'sectorDisp', 'longBusinessSummary', 'fullTimeEmployees', 'companyOfficers', 'compensationAsOfEpochDate', 'executiveTeam', 'maxAge', 'priceHint', 'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose', 'regularMarketOpen', 'regularMarketDayLow', 'regularMarketDayHigh', 'dividendRate', 'dividendYield', 'exDividendDate', 'payoutRatio', 'fiveYearAvgDividendYield', 'beta', 'trailingPE', 'volume', 'regularMarketVolume', 'averageVolume', 'averageVolume10days', 'averageDailyVolume10Day', 'bid', 'ask', 'bidSize', 'askSize', 'marketCap', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 'fiftyDayAverage', 'twoHundredDayAverage', 'trailingAnnualDividendRate', 'trailingAnnualDividendYield', 'currency', 'tradeable', 'enterpriseValue', 'profitMargins', 'floatShares', 'sharesOutstanding', 'heldPercentInsiders', 'heldPer

In [59]:
yf.Ticker('AAPL').balancesheet

,2024-09-30,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Treasury Shares Number,NaN,0.0,NaN,NaN,NaN
Ordinary Shares Number,15116786000.0,15550061000.0,15943425000.0,16426786000.0,NaN
Share Issued,15116786000.0,15550061000.0,15943425000.0,16426786000.0,NaN
Net Debt,76686000000.0,81123000000.0,96423000000.0,89779000000.0,NaN
Total Debt,106629000000.0,111088000000.0,132480000000.0,136522000000.0,NaN
...,...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,65171000000.0,61555000000.0,48304000000.0,62639000000.0,NaN
Other Short Term Investments,35228000000.0,31590000000.0,24658000000.0,27699000000.0,NaN
Cash And Cash Equivalents,29943000000.0,29965000000.0,23646000000.0,34940000000.0,NaN
Cash Equivalents,2744000000.0,1606000000.0,5100000000.0,17635000000.0,NaN


In [60]:
yf.Ticker('AAPL').cashflow

,2024-09-30,2023-09-30,2022-09-30,2021-09-30,2020-09-30
Free Cash Flow,108807000000.0,99584000000.0,111443000000.0,92953000000.0,NaN
Repurchase Of Capital Stock,-94949000000.0,-77550000000.0,-89402000000.0,-85971000000.0,NaN
Repayment Of Debt,-9958000000.0,-11151000000.0,-9543000000.0,-8750000000.0,NaN
Issuance Of Debt,0.0,5228000000.0,5465000000.0,20393000000.0,NaN
Issuance Of Capital Stock,NaN,NaN,NaN,1105000000.0,880000000.0
Capital Expenditure,-9447000000.0,-10959000000.0,-10708000000.0,-11085000000.0,NaN
Interest Paid Supplemental Data,NaN,3803000000.0,2865000000.0,2687000000.0,3002000000.0
Income Tax Paid Supplemental Data,26102000000.0,18679000000.0,19573000000.0,25385000000.0,NaN
End Cash Position,29943000000.0,30737000000.0,24977000000.0,35929000000.0,NaN
Beginning Cash Position,30737000000.0,24977000000.0,35929000000.0,39789000000.0,NaN


In [61]:
yf.Ticker('AAPL').history(period='10y')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2015-03-09 00:00:00-04:00,28.577011,28.936571,27.929361,28.393883,354114000,0.0,0.0
2015-03-10 00:00:00-04:00,28.230851,28.411746,27.647966,27.806528,275426400,0.0,0.0
2015-03-11 00:00:00-04:00,27.860129,27.864595,27.270544,27.299576,275756000,0.0,0.0
2015-03-12 00:00:00-04:00,27.315198,27.893618,27.163336,27.793119,193450800,0.0,0.0
2015-03-13 00:00:00-04:00,27.781965,28.005293,27.375509,27.601068,207309200,0.0,0.0
...,...,...,...,...,...,...,...
2025-03-03 00:00:00-05:00,241.789993,244.029999,236.110001,238.029999,47184000,0.0,0.0
2025-03-04 00:00:00-05:00,237.710007,240.070007,234.679993,235.929993,53798100,0.0,0.0
2025-03-05 00:00:00-05:00,235.419998,236.550003,229.229996,235.740005,47227600,0.0,0.0


In [94]:
cashflow = yf.Ticker('AAPL').cashflow
earning_dates = cashflow.columns.tolist()
print(earning_dates)

yf.Ticker('AAPL').history(period='10y')

[Timestamp('2024-09-30 00:00:00'), Timestamp('2023-09-30 00:00:00'), Timestamp('2022-09-30 00:00:00'), Timestamp('2021-09-30 00:00:00'), Timestamp('2020-09-30 00:00:00')]


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2015-03-09 00:00:00-04:00,28.577006,28.936565,27.929355,28.393877,354114000,0.0,0.0
2015-03-10 00:00:00-04:00,28.230857,28.411752,27.647971,27.806534,275426400,0.0,0.0
2015-03-11 00:00:00-04:00,27.860125,27.864591,27.270540,27.299572,275756000,0.0,0.0
2015-03-12 00:00:00-04:00,27.315206,27.893626,27.163343,27.793127,193450800,0.0,0.0
2015-03-13 00:00:00-04:00,27.781963,28.005291,27.375507,27.601067,207309200,0.0,0.0
...,...,...,...,...,...,...,...
2025-03-03 00:00:00-05:00,241.789993,244.029999,236.110001,238.029999,47184000,0.0,0.0
2025-03-04 00:00:00-05:00,237.710007,240.070007,234.679993,235.929993,53798100,0.0,0.0
2025-03-05 00:00:00-05:00,235.419998,236.550003,229.229996,235.740005,47227600,0.0,0.0
